In [18]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import sklearn.model_selection
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

#word2vec
from gensim.models import Word2Vec  
import gensim
import string



from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D

[nltk_data] Downloading package wordnet to /home/typedoor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#The path tht our data is stored
PATH = '/home/typedoor/PLBRS/Data/'
sent_tokenize?

Signature: sent_tokenize(text, language='english')
Docstring:
Return a sentence-tokenized copy of *text*,
using NLTK's recommended sentence tokenizer
(currently :class:`.PunktSentenceTokenizer`
for the specified language).

:param text: text to split into sentences
:param language: the model name in the Punkt corpus
File:      ~/anaconda3/envs/umc/lib/python3.6/site-packages/nltk/tokenize/__init__.py
Type:      function


In [41]:
#load the data, select only important characters, tokenize the corpus
sp = pd.read_csv(PATH + 'south_park.csv')
sp_clean = sp[(sp.Character == 'Cartman') | (sp.Character == 'Kyle')].reset_index(drop = True)
sp_clean.Line = sp_clean.Line.apply(sent_tokenize)
sp_clean.Line[0]

['Going away?', 'For how long?']

In [65]:
stop_words = set(stopwords.words('english'))

ps = PorterStemmer()
lem = WordNetLemmatizer()

def remove_stop_words(line):
    filtered_line = []
    for word in line:
        if word not in stop_words:
            filtered_line.append(word)
    
    return filtered_line
    #return " ".join(filtered_line)

def stemm_words(line):
    stemmed_line = []
    
    for word in line:
        stemmed_line.append(ps.stem(word))
    
    return stemmed_line
    #return "".join(stemmed_line)

def lemmatize_words(line):
    lemmed_line = []
    
    for word in line:
        lemmed_line.append(lem.lemmatize(word))
     
    return lemmed_line

    #return "".join(lemmed_line)

In [66]:
#removing stop words, and lemmatizing
sp_clean.Line = sp_clean.Line.apply(remove_stop_words)
sp_clean.Line = sp_clean.Line.apply(stemm_words)
sp_clean.Line = sp_clean.Line.apply(lemmatize_words)

type(sp_clean.Line[0])

list

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(sp_clean['Line'])


In [14]:
y = sp_clean.Character.values

X_train, X_test, y_train, y_test = train_test_split(text_counts, y, test_size = 0.33, random_state = 53, stratify = y)
count_vectorizer = CountVectorizer(stop_words = 'english',ngram_range=(1,3))

#count_train = count_vectorizer.fit_transform(X_train)
#count_test = count_vectorizer.transform(X_test)

nb_classifier = MultinomialNB()

# Fit the classifier to the training data
nb_classifier.fit(X_train,y_train)

# Create the predicted tags: pred
pred = nb_classifier.predict(X_test)

# Calculate the accuracy score: score
score = metrics.accuracy_score(y_test,pred)
print(score)

cm = metrics.classification_report(y_test, pred,labels = ['Kyle' ,'Cartman'])
confusion = metrics.confusion_matrix(y_test, pred, labels = ['Kyle', 'Cartman'])
print(cm)
print(confusion)

0.6678039145268451
              precision    recall  f1-score   support

        Kyle       0.63      0.51      0.56      2343
     Cartman       0.69      0.78      0.73      3226

   micro avg       0.67      0.67      0.67      5569
   macro avg       0.66      0.65      0.65      5569
weighted avg       0.66      0.67      0.66      5569

[[1199 1144]
 [ 706 2520]]


**WORD2VEC**

In [96]:
sp_clean.Line = [line.lower().split() for line in sp_clean.copy().Line]
w2v_model=Word2Vec(sp_clean.Line.values,size=20,min_count=3,window=3,sg=1,hs=0,seed=42,workers=8)

In [97]:
w2v_model.train(sp_clean.Line,total_examples=len(sp_clean.Line),epochs=5)

(447144, 570350)

In [98]:
w2v_model.wv.get_vector('room')

array([-0.15155533, -0.01953503, -0.01645868, -0.07574271, -0.02216   ,
        0.48130372,  0.35574567, -0.20575455, -0.6798989 ,  0.8254573 ,
       -0.31124097,  0.66434574, -0.35054132, -0.10033735, -0.01128355,
       -0.4839375 ,  0.44111776,  0.19567357, -0.17299318,  0.42479464],
      dtype=float32)

In [100]:
w2v_model.wv.most_similar('government')

[('be,', 0.9895018339157104),
 ('mind,', 0.9893526434898376),
 ('playstation', 0.9892599582672119),
 ('trick', 0.9890922904014587),
 ('happen', 0.988867998123169),
 ('alone', 0.988655686378479),
 ('belongs', 0.9883878827095032),
 ('men', 0.9882310628890991),
 ('needed', 0.9881280660629272),
 ('beating', 0.9880689978599548)]

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
text_tf= tf.fit_transform(sp_clean['Line'])

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_tf, y, test_size=0.3, random_state=123)

In [32]:
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
confusion = metrics.confusion_matrix(y_test, predicted, labels = ['Kyle', 'Cartman'])
cm = metrics.classification_report(y_test, predicted,labels = ['Kyle' ,'Cartman'])
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))
print(confusion)
print(cm)

MultinomialNB Accuracy: 0.6683129197945477
[[ 740 1397]
 [ 282 2643]]
              precision    recall  f1-score   support

        Kyle       0.72      0.35      0.47      2137
     Cartman       0.65      0.90      0.76      2925

   micro avg       0.67      0.67      0.67      5062
   macro avg       0.69      0.62      0.61      5062
weighted avg       0.68      0.67      0.64      5062



In [34]:
clf = RandomForestClassifier(n_jobs = -1, n_estimators = 7500, bootstrap = True, max_depth = 2,class_weight= 'balanced_subsample').fit(X_train,y_train)
predicted= clf.predict(X_test)
print("Random Forest Accuracy:",metrics.accuracy_score(y_test, predicted))
confusion = metrics.confusion_matrix(y_test, predicted, labels = ['Kyle', 'Cartman'])
cm = metrics.classification_report(y_test, predicted,labels = ['Kyle' ,'Cartman'])
print(confusion)
print(cm)

Random Forest Accuracy: 0.6467799288818649
[[1614  523]
 [1265 1660]]
              precision    recall  f1-score   support

        Kyle       0.56      0.76      0.64      2137
     Cartman       0.76      0.57      0.65      2925

   micro avg       0.65      0.65      0.65      5062
   macro avg       0.66      0.66      0.65      5062
weighted avg       0.68      0.65      0.65      5062



In [35]:
clf = LogisticRegression().fit(X_train,y_train)
predicted= clf.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
confusion = metrics.confusion_matrix(y_test, predicted, labels = ['Kyle', 'Cartman'])
cm = metrics.classification_report(y_test, predicted,labels = ['Kyle' ,'Cartman'])
print(confusion)
print(cm)

/home/typedoor/anaconda3/envs/umc/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy: 0.6977479257210588
[[1086 1051]
 [ 479 2446]]
              precision    recall  f1-score   support

        Kyle       0.69      0.51      0.59      2137
     Cartman       0.70      0.84      0.76      2925

   micro avg       0.70      0.70      0.70      5062
   macro avg       0.70      0.67      0.67      5062
weighted avg       0.70      0.70      0.69      5062



In [39]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

kernels = ['linear','rbf','sigmoid','poly']
C = [0.001, 0.05, 0.1, 0.25, 0.5, 1, 1.5, 2, 3, 5]


param_grid = {'kernel' : kernels,'shrinking' : [True, False], 'probability' : [True, False], 'class_weight' : ['balanced']}

clf = SVC()

svc_cv = GridSearchCV(clf,param_grid,cv = 5)
svc_cv.fit(X_train,y_train)

print("Tuned Logistic Regression Parameters: {}".format(svc_cv.best_params_)) 
print("Best score is {}".format(svc_cv.best_score_))


'''
predicted= clf.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
confusion = metrics.confusion_matrix(y_test, predicted, labels = ['Kyle', 'Cartman'])
cm = metrics.classification_report(y_test, predicted,labels = ['Kyle' ,'Cartman'])
print(confusion)
print(cm)'''

/home/typedoor/anaconda3/envs/umc/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Tuned Logistic Regression Parameters: {'class_weight': 'balanced', 'kernel': 'linear', 'probability': True, 'shrinking': False}
Best score is 0.6734400135466938


'\npredicted= clf.predict(X_test)\nprint("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))\nconfusion = metrics.confusion_matrix(y_test, predicted, labels = [\'Kyle\', \'Cartman\'])\ncm = metrics.classification_report(y_test, predicted,labels = [\'Kyle\' ,\'Cartman\'])\nprint(confusion)\nprint(cm)'